<a href="https://colab.research.google.com/github/shaafi56/-Classifying-Somali-News-into-International-and-Local-Categories-using-NLP-Text-Classification/blob/main/update_three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
!pip install WordCloud
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
from docx import Document

# Assuming the uploaded file is named 'example.docx'
doc = Document('datasets.docx')

# Extract text from each paragraph in the document
text = []
for paragraph in doc.paragraphs:
    text.append(paragraph.text)

# Print the extracted text
for line in text:
    print(line)


In [ ]:
# Word Cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(text))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Word Cloud for News Text')
plt.axis('off')

In [88]:
# Create a DataFrame from the list of text
text_df = pd.DataFrame({'Text': text})
# Tokenize the text in the DataFrame
text_df['Tokenized_Text'] = text_df['Text'].apply(lambda x: word_tokenize(x))


In [90]:
# Stopword Removal
#stop_words = ['waa', 'waana', 'waan', 'ah', 'oo', 'ee', 'waxey', 'ugu', 'waxa','eey', 'iyo', 'aad', 'baan', 'u', 'leh', 'beey', 'ahna', 'tiq', 'sidoo', 'kalena', 'laakiin','ay','is','ayaa', 'isku', 'ka',  'ku', 'lagu', 'ayay',  'ayuu', 'inuu', 'inaa', 'si', 'laga', 'in', 'haya', 'haaya', 'intii', 'uu', 'ii', 'la', 'looga', 'Mar', 'kaliya', 'waxaa', 'loo']
stop_words = ['waa', 'waana', 'waan', 'ah', 'oo', 'ee', 'waxey', 'ugu', 'waxa',
             'eey', 'iyo', 'aad', 'baan', 'u', 'leh', 'yar', 'yaraaday', 'yarayd', 'yeesheen', 'yeeshee', 'beey', 'ahna', 'tiq', 'oo', 'yimid', 'yiri' 'Ku','ee' 'sidoo', 'kalena', 'laakiin',
             'yeeshay','yesheen', 'yaala', 'ay', 'yaabaa', 'yaabo', 'is', 'yare', 'yaab', 'yeesho', 'yareeyo','yeelatay','ayaa','isku', 'ka', 'ku', 'yiri','yimaadaan','yimaaday','yimaaday','yihiin', 'lagu', 'ayay',  'ayuu',
              'inuu', 'inaa', 'si', 'laga', 'in', 'yaryar','yaalla', 'yaal','yaalay', 'haya', 'haaya', 'intii', 'uu', 'ii', 'la', 'looga','mar','kaliya','loo','yahay','sida','soo']
text_df['Filtered_Text'] = text_df['Tokenized_Text'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
stopWords = list(stop_words)

In [89]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')
docs = nlp(text_df['Text'].str.cat(sep=' '))

In [ ]:
from string import punctuation
punctuation = punctuation + '\n'
punctuation

In [ ]:
#words frequency are how many words are repetition
space = '\n '
word_frequencies = {}
for word in docs:
  if word.text.lower() not in stop_words:
    if word.text.lower() not in punctuation:
      if word.text not in word_frequencies.keys():
        if word.text not in space:
          word_frequencies[word.text] = 1

      else:
        word_frequencies[word.text] += 1
#display as a list with word frequencies
for key, value in word_frequencies.items():
  print(key, value)

In [101]:
#which word that is maximum repetition
max_frequencies = max(word_frequencies.values())
print(max_frequencies)

14


In [ ]:
for word in word_frequencies.keys():
  word_frequencies[word] = (word_frequencies[word]/max_frequencies)
  print(word, word_frequencies[word])

In [ ]:
sent_token = [sent.text for sent in docs.sents]
sent_token

In [131]:
sent_score = {}
for sent in sent_token:
  for word in sent.split():
    if word.lower() in word_frequencies.keys():
      if sent not in sent_score.keys():
        sent_score[sent] = word_frequencies[word]
      else:
        sent_score[sent] += word_frequencies[word]
sent_score

{'Nin u dhashay dalka Taliyaaniga oo la nool xanuunka': 18,
 'HIV ayaa lagu xukumay xabsi dhan afar iyo labaatan sano kadib markii lagu helay dembi ah in si ula kac ah uu xanunkaasi uu u qaadsiiyay sodon haween ah oo u dhashay dalkaasi\n Valentino Talluto ayaa la xiray bishii November 2015 Sodon iyo saddex jirkan lagu magacaabo Valentino Tallut oo ahaa xisaabiye ayaa lagu eedeeyay in haweenka uu la hasaawo uu qaadsiin jiray xanuunka kadib markii uu ogaaday in uu qabo xanuunka AIDSka\n Baarayaasha kiiskani ayaa sheegay in ninkani uu sidoo kale xanuunka qaadsiiyay caruurta ay caloosha ku sideen qaar kamid ah hawwenkaasi waxaan qareenka difaacayay Valentino Talluto uu sheegay in falkani uusan aheyn mid ulakac ah\n Suaal xisaab ah oo aan la xalin karin oo la weeydiiyay ardayda Shiinaha\n Dadka isticmaala baraha bulshada dalka Shiinaha ayaa dhaleeceeyay suaal ku jirtay imtixaam arday laga qaadayay taas oo aysanba suurtogal ahayn in la xaliyo\n Suaasha oo la weeydiiyay ardayda dhigata fasalk

In [ ]:
import pandas as pd
df = pd.DataFrame(list(sent_score.items()), columns=['Sentence', 'Score'])
df

In [143]:
from heapq import nlargest
num_sentences = 2
#as you split the sentences
best_sentences = sorted(sent_score.items(), key=lambda x: x[1], reverse=True)[:num_sentences]
best_sentences
# as you jion in one parigraph
# nlargest(num_sentences, sent_score, key=sent_score.get)


[('oo lagu saleynayo heshiiskii 17kii September\n Madaxweyne Maxamed Cabdullaahi Farmaajo ayaa khudbadiisa maanta ku sheegay in xeerkii Golaha Shacabku ansixiyeen ee muddada loogu kordhinayay hayaddaha dowladda uu ahaa fursad dalka looga badbaadinayay dib u dhac iyo hubanti laaan dalka la soo darista\n Wuxuu sheegay in sharcigaas ay ka hor yimaadaan qaar ka mid ah dowladda goballeedyada isla markaana magaalada Muqdisho ee caasimadda Soomaaliya uu ka abuuray cakirnaan iyo qalalaase\n Magaalada Muqdisho waxaa laga abuuray qalalaasexasarado qorsheysan waxaa la barakiciyay shacab waxaana khatar la galiyay naftooda iyo hantidooda ayuu yiri madaxweynaha Soomaaliya\n Lama oggalaan karo mar labaad Maxamaed Cabdullaahi Farmaajo ayaa sheegay in magaalada Muqdisho maadama ay tahay caasimadda umadda Soomaaliyeed aan la oggolaan doonin inay dib ugu laabato dhibaaooyinkii ay soo martay soddankii sano ee dalka Soomaaliya uu burburku ka jiray\n ',
  201),
 ('HIV ayaa lagu xukumay xabsi dhan afar iyo l